<a href="https://colab.research.google.com/github/loicbi/Databricks/blob/develop/15_Databricks%7C_Spark_%7C_Pyspark_%7C_Read_Json%7C_Flatten_Json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install findspark

In [0]:
!pip install pyspark

In [0]:
import findspark
from pyspark.sql.functions import  * # to_date, col, split, char_length, lit, trim
# findspark.init()
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder.appName("app_name").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


df = spark.read.json("/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/US_category_id.json", multiLine=True)
# df = spark.read.json("/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/product.json", multiLine=True)
df.printSchema()
df.show(10)


In [0]:

#Flatten array of structs and structs
def flatten_df(df):
  #  compute Complex Fields (Lists and Structs) in Schema
   complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   while len(complex_fields)!=0:
      col_name=list(complex_fields.keys())[0]
      print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))

      # if StructType then convert all sub element to columns.
      # i.e. flatten structs
      if (type(complex_fields[col_name]) == StructType):
         expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
         df=df.select("*", *expanded).drop(col_name)

      # if ArrayType then add the Array Elements as Rows using the explode function
      # i.e. explode Arrays
      elif (type(complex_fields[col_name]) == ArrayType):
         df=df.withColumn(col_name,explode_outer(col_name))

      # recompute remaining Complex Fields in Schema
      complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   return df

In [0]:
df_flatten = flatten_df(df)

In [0]:
df_flatten.show(5)

## json from link

In [0]:
from urllib.request import urlopen
import findspark
import json
from pyspark.sql.functions import  * # to_date, col, split, char_length, lit, trim
# findspark.init()
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder.appName("app_name").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


#Flatten array of structs and structs
def flatten_df_all(df):
   i_column_exist = 0
  #  compute Complex Fields (Lists and Structs) in Schema
   complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   while len(complex_fields)!=0:
      col_name=list(complex_fields.keys())[0]
      # print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))

      # if StructType then convert all sub element to columns.
      # i.e. flatten structs
      if (type(complex_fields[col_name]) == StructType):
         expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
         df=df.select("*", *expanded).drop(col_name)

      # if ArrayType then add the Array Elements as Rows using the explode function
      # i.e. explode Arrays
      elif (type(complex_fields[col_name]) == ArrayType):
          # check if colNameList [] contains StructType ==> [{}]
          print(isinstance(df.schema[col_name].dataType.elementType, StructType))
          df.select(col_name).show()
          # if isinstance(df.schema[col_name].dataType.elementType, StructType) is True:
          if col_name in df.columns:
            df=df.withColumnRenamed(col_name,col_name+'_'+str(i_column_exist)).withColumn(col_name+'_'+str(i_column_exist),explode_outer(col_name+'_'+str(i_column_exist)))
          else:
            df=df.withColumn(col_name,explode_outer(col_name))


      # recompute remaining Complex Fields in Schema
      complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   i_column_exist += 1
   return df

# method from link
# url = 'https://world.openfoodfacts.org/api/v2/product/737628064502.json'
# jsonData = urlopen(url).read().decode('utf-8')
# rdd = spark.sparkContext.parallelize([jsonData])
# df_link = spark.read.json(rdd, multiLine=True)

# method from drive
df_product_2 = spark.read.json("/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/product2.json", multiLine=True)

# Create outer method to return the flattened Data Frame
def flatten_json_df(_df: DataFrame) -> DataFrame:
    # List to hold the dynamically generated column names
    flattened_col_list = []

    # Inner method to iterate over Data Frame to generate the column list
    def get_flattened_cols(df: DataFrame, struct_col: str = None) -> None:
        for col in df.columns:
            # print(col, df.schema[col].dataType.typeName(), df.schema[col].dataType)

            if df.schema[col].dataType.typeName() != 'struct':
                if (isinstance(df.schema[col].dataType, ArrayType)):
                  if (isinstance(df.schema[col].dataType.elementType, StructType)):
                    array_struct_col = col
                    # print(col, 'struct_col:', struct_col)

                    for col_name in (json.loads(df.schema[col].dataType.json())['elementType']['fields']):
                        # for c in json.loads(df.schema[col].dataType.json())['elementType']['fields'][n]['name']:
                        # print(col_name['name'])
                        col_child = col_name['name']
                        array_chained_col = array_struct_col +"."+ col_child if array_struct_col is not None else col
                        # get_flattened_cols(df.select(col_child+".*"), col)
                        print(array_chained_col)



                        # array_t = struct_col + "." + col
                        # flattened_col_list.append(f"{t} as {t.replace('.','_')}")



                    # if ('type' in (json.loads(df.schema[col].dataType.json()))):
                    #   print(col, 'contains structure')
                    # else:
                    #   print(col, 'doesn\'t contains structure')



                if struct_col is None:
                    flattened_col_list.append(f"{col} as {col.replace('.','_')}")
                else:
                    t = struct_col + "." + col
                    flattened_col_list.append(f"{t} as {t.replace('.','_')}")
                    print('not struc', t)
            else:
                chained_col = struct_col +"."+ col if struct_col is not None else col
                print('struct', chained_col)
                get_flattened_cols(df.select(col+".*"), chained_col)

    # Call the inner Method
    get_flattened_cols(_df)

    # Return the flattened Data Frame
    return _df.selectExpr(flattened_col_list)# Generate the flattened DF

# df.printSchema()
df_link_flatten = flatten_json_df(df_product_2)
df_link_flatten.show()
# df_link = spark.read.json('https://world.openfoodfacts.org/api/v2/product/737628064502.json')

In [0]:
import json


data = r'''
{
 "kind": "youtube#videoCategoryListResponse",
 "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM\"",
 "items": [
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ\"",
   "id": "1",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Film & Animation",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA\"",
   "id": "2",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Autos & Vehicles",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg\"",
   "id": "10",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Music",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI\"",
   "id": "15",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Pets & Animals",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM1XVQ9zbGec\"",
   "id": "17",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Sports",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/FJwVpGCVZ1yiJrqZbpqe68Sy_OE\"",
   "id": "18",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Short Movies",
    "assignable": false
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf_DkLN8CouA\"",
   "id": "19",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Travel & Events",
    "assignable": true
   }
  }
 ]
} '''
data = json.loads(data)

data

## PySpark — Flatten JSON/Struct Data Frame dynamically
### Example variable local

[JSON/Struct Data Frame dynamically](https://subhamkharwal.medium.com/pyspark-flatten-json-struct-data-frame-dynamically-c2e5d8937dcc)

In [0]:
# Lets create an Example Data Frame to hold JSON data
# Example Data Frame with column having JSON data
_data = [
    ['EMP001', '{"dept" : "account", "fname": "Ramesh", "lname": "Singh", "skills": ["excel", "tally", "word"]}'],
    ['EMP002', '{"dept" : "sales", "fname": "Siv", "lname": "Kumar", "skills": ["biking", "sales"]}'],
    ['EMP003', '{"dept" : "hr", "fname": "MS Raghvan", "skills": ["communication", "soft-skills"], "hobbies" : {"cycling": "expert", "computers":"basic"}}'],
    ['EMP003', '{"dept" : "hr", "fname": "MS Raghvan", "skills": ["communication", "soft-skills"], "hobbies" : {"cycling": "expert", "computers":"basic"}}']
]

# Columns for the data
_cols = ['emp_no', 'raw_data']

# Lets create the raw Data Frame
df_raw = spark.createDataFrame(data = _data, schema = _cols)

# Determine the schema of the JSON payload from the column
json_schema_df = spark.read.json(df_raw.rdd.map(lambda row: row.raw_data))
json_schema = json_schema_df.schema

# Apply the schema to payload to read the data
from pyspark.sql.functions import from_json
df_details = df_raw.withColumn("emp_details", from_json(df_raw["raw_data"], json_schema)).drop("raw_data")
df_details.show(10, False)

df_details.printSchema()

In [0]:
# Python function to flatten the data dynamically
from pyspark.sql import DataFrame
# Create outer method to return the flattened Data Frame
def flatten_json_df(_df: DataFrame) -> DataFrame:
    # List to hold the dynamically generated column names
    flattened_col_list = []

    # Inner method to iterate over Data Frame to generate the column list
    def get_flattened_cols(df: DataFrame, struct_col: str = None) -> None:
        for col in df.columns:
            if df.schema[col].dataType.typeName() != 'struct':
                if struct_col is None:
                    flattened_col_list.append(f"{col} as {col.replace('.','_')}")
                else:
                    t = struct_col + "." + col
                    flattened_col_list.append(f"{t} as {t.replace('.','_')}")
            else:
                chained_col = struct_col +"."+ col if struct_col is not None else col
                get_flattened_cols(df.select(col+".*"), chained_col)

    # Call the inner Method
    get_flattened_cols(_df)

    # Return the flattened Data Frame
    return _df.selectExpr(flattened_col_list)# Generate the flattened DF
flattened_df = flatten_json_df(df_details)
flattened_df.show(10)
# Print Schema
flattened_df.printSchema()

In [0]:
import json

data = '''
{"customer_id":45721,"firstname":"Tiffany","lastname":"Skinner","salutation":"Miss","gender":"F","birthdate":"1935-09-13","birth_country":"RWANDA","address_id":2133,"email_address":"Tiffany.Skinner@J.edu","is_preffered_customer":"N","demographics":{"income_range":[50001,60000],"buy_potential":">10000","purchase_estimate":1000,"vehicle_count":2,"credit_rating":"High Risk","education_status":"Secondary"}}
{"customer_id":72411,"firstname":"Dennis","lastname":"Pettit","salutation":"Mr.","gender":"F","birthdate":"1973-11-09","birth_country":"MALAWI","address_id":25398,"email_address":"Dennis.Pettit@PtS7Fbdax.com","is_preffered_customer":"Y","demographics":{"income_range":[170001,180000],"buy_potential":"0-500","purchase_estimate":3000,"vehicle_count":4,"credit_rating":"Unknown","education_status":"Primary"}}
{"customer_id":50358,"firstname":"Issac","lastname":"Saunders","salutation":"Mr.","gender":"F","birthdate":"1979-03-30","birth_country":"JAMAICA","address_id":23242,"email_address":"Issac.Saunders@Ykkz.edu","is_preffered_customer":"Y","demographics":{"income_range":[120001,130000],"buy_potential":"0-500","purchase_estimate":2500,"vehicle_count":3,"credit_rating":"Unknown","education_status":"Unknown"}}
{"customer_id":96143,"firstname":"Carl","lastname":"Mitchell","salutation":"Dr.","gender":"F","birthdate":"1966-10-16","birth_country":"IRELAND","address_id":37550,"email_address":"Carl.Mitchell@iqios5dGzoAkgSfdkO.org","is_preffered_customer":"Y","demographics":{"income_range":[20001,30000],"buy_potential":"Unknown","purchase_estimate":1000,"vehicle_count":-1,"credit_rating":"High Risk","education_status":"College"}}
{"customer_id":93545,"firstname":"Antonio","lastname":"Lyon","salutation":"Dr.","gender":"F","birthdate":"1953-05-24","birth_country":"PAPUA NEW GUINEA","address_id":22369,"email_address":"Antonio.Lyon@Alqrvh.org","is_preffered_customer":"Y","demographics":{"income_range":[180001,190000],"buy_potential":"1001-5000","purchase_estimate":8000,"vehicle_count":1,"credit_rating":"Unknown","education_status":"4 yr Degree"}}
{"customer_id":92534,"firstname":"Jesse","lastname":"Alexander","salutation":"Dr.","gender":"M","birthdate":"1968-10-22","birth_country":"ZIMBABWE","address_id":7628,"email_address":"Jesse.Alexander@OU.edu","is_preffered_customer":"N","demographics":{"income_range":[120001,130000],"buy_potential":">10000","purchase_estimate":8000,"vehicle_count":2,"credit_rating":"Unknown","education_status":"Advanced Degree"}}
{"customer_id":91798,"firstname":"Philip","lastname":"Miller","salutation":"Mr.","gender":"M","birthdate":"1934-08-05","birth_country":"DJIBOUTI","address_id":24321,"email_address":"Philip.Miller@1j1uLUoeT3N.edu","is_preffered_customer":"N","demographics":{"income_range":[150001,160000],"buy_potential":"5001-10000","purchase_estimate":500,"vehicle_count":3,"credit_rating":"High Risk","education_status":"Unknown"}}
{"customer_id":82953,"firstname":"Ronald","lastname":"Harris","salutation":"Sir","gender":"M","birthdate":"1958-08-02","birth_country":"LESOTHO","address_id":49705,"email_address":"Ronald.Harris@KUjp3Gqoqt.com","is_preffered_customer":"N","demographics":{"income_range":[20001,30000],"buy_potential":"501-1000","purchase_estimate":6000,"vehicle_count":4,"credit_rating":"Good","education_status":"4 yr Degree"}}
{"customer_id":70926,"firstname":"Myrtle","lastname":"Wolford","salutation":"Dr.","gender":"F","birthdate":"1983-11-14","birth_country":"C�TE D'IVOIRE","address_id":36381,"email_address":"Myrtle.Wolford@lbKOsj.com","is_preffered_customer":"Y","demographics":{"income_range":[110001,120000],"buy_potential":"Unknown","purchase_estimate":2500,"vehicle_count":0,"credit_rating":"Unknown","education_status":"2 yr Degree"}}
{"customer_id":12157,"firstname":"William","lastname":"Daniel","salutation":"Mr.","gender":"F","birthdate":"1976-03-27","birth_country":"BANGLADESH","address_id":40974,"email_address":"William.Daniel@Ri4LeFs5lB4QpUfF.edu","is_preffered_customer":"N","demographics":{"income_range":[170001,180000],"buy_potential":">10000","purchase_estimate":6000,"vehicle_count":2,"credit_rating":"Unknown","education_status":"Primary"}}
{"customer_id":65983,"firstname":"Perla","lastname":"Edwards","salutation":"Dr.","gender":"F","birthdate":"1947-02-27","birth_country":"LIBERIA","address_id":3282,"email_address":"Perla.Edwards@nvR7A3h9o.edu","is_preffered_customer":"Y","demographics":{"income_range":[180001,190000],"buy_potential":"0-500","purchase_estimate":9500,"vehicle_count":1,"credit_rating":"Good","education_status":"Advanced Degree"}}
{"customer_id":36493,"firstname":"Jeffrey","lastname":"Clary","salutation":"Mr.","gender":"F","birthdate":"1924-02-17","birth_country":"ROMANIA","address_id":1289,"email_address":"Jeffrey.Clary@lBvUIdcnhNFey.com","is_preffered_customer":"N","demographics":{"income_range":[40001,50000],"buy_potential":"0-500","purchase_estimate":2500,"vehicle_count":1,"credit_rating":"Low Risk","education_status":"2 yr Degree"}}
{"customer_id":41148,"firstname":"Charles","lastname":"Patterson","salutation":"Mr.","gender":"M","birthdate":"1961-08-12","birth_country":"DOMINICA","address_id":29846,"email_address":"Charles.Patterson@bHiA5JcaZzpS89L8iL.edu","is_preffered_customer":"N","demographics":{"income_range":[120001,130000],"buy_potential":"0-500","purchase_estimate":10000,"vehicle_count":1,"credit_rating":"Unknown","education_status":"Advanced Degree"}}
{"customer_id":61969,"firstname":"Nikki","lastname":"Watson","salutation":"Miss","gender":"M","birthdate":"1977-08-15","birth_country":"PARAGUAY","address_id":22881,"email_address":"Nikki.Watson@DNC.org","is_preffered_customer":"Y","demographics":{"income_range":[90001,100000],"buy_potential":"1001-5000","purchase_estimate":6500,"vehicle_count":0,"credit_rating":"High Risk","education_status":"Advanced Degree"}}
{"customer_id":69480,"firstname":"Marion","lastname":"Paul","salutation":"Sir","gender":"M","birthdate":"1955-10-11","birth_country":"SAUDI ARABIA","address_id":42838,"email_address":"Marion.Paul@tfyhY8prgDTSdZ.org","is_preffered_customer":"N","demographics":{"income_range":[100001,110000],"buy_potential":"5001-10000","purchase_estimate":3000,"vehicle_count":0,"credit_rating":"Unknown","education_status":"Unknown"}}
{"customer_id":4739,"firstname":"Gregory","lastname":"Martin","salutation":"Mr.","gender":"M","birthdate":"1958-03-15","birth_country":"UGANDA","address_id":2625,"email_address":"Gregory.Martin@GI78ZP8x.edu","is_preffered_customer":"Y","demographics":{"income_range":[130001,140000],"buy_potential":"1001-5000","purchase_estimate":5500,"vehicle_count":4,"credit_rating":"Good","education_status":"Unknown"}}
{"customer_id":67634,"salutation":"Sir","gender":"F","address_id":37149,"is_preffered_customer":"Y","demographics":{"income_range":[140001,150000],"buy_potential":"Unknown","purchase_estimate":9000,"vehicle_count":1,"credit_rating":"High Risk","education_status":"Primary"}}
{"customer_id":87143,"firstname":"Corazon","lastname":"Martinez","salutation":"Miss","gender":"F","birthdate":"1984-08-13","birth_country":"SAMOA","address_id":27621,"email_address":"Corazon.Martinez@BrDtDnXnkci1Pp2an.edu","is_preffered_customer":"N","demographics":{"income_range":[30001,40000],"buy_potential":"5001-10000","purchase_estimate":9000,"vehicle_count":4,"credit_rating":"High Risk","education_status":"Unknown"}}
{"customer_id":64173,"firstname":"Christopher","lastname":"Byrd","salutation":"Sir","gender":"M","birthdate":"1936-04-10","birth_country":"MALDIVES","address_id":21565,"email_address":"Christopher.Byrd@bLMxmInbyn.org","is_preffered_customer":"Y","demographics":{"income_range":[20001,30000],"buy_potential":"0-500","purchase_estimate":8500,"vehicle_count":4,"credit_rating":"Good","education_status":"2 yr Degree"}}
{"customer_id":39412,"firstname":"Constance","lastname":"Seaman","salutation":"Miss","gender":"F","birthdate":"1980-03-10","birth_country":"AUSTRALIA","address_id":18881,"email_address":"Constance.Seaman@GItsENvPOs7C.org","is_preffered_customer":"N","demographics":{"income_range":[40001,50000],"buy_potential":"5001-10000","purchase_estimate":3000,"vehicle_count":2,"credit_rating":"High Risk","education_status":"4 yr Degree"}}
{"customer_id":39412,"firstname":"Constance","lastname":"Seaman","salutation":"Miss","gender":"F","birthdate":"1980-03-10","birth_country":"AUSTRALIA","address_id":18881,"email_address":"Constance.Seaman@GItsENvPOs7C.org","is_preffered_customer":"N","demographics":{"income_range":[40001,50000],"buy_potential":"5001-10000","purchase_estimate":3000,"vehicle_count":2,"credit_rating":"High Risk","education_status":"4 yr Degree"}}
'''

import json
from pyspark.sql import SparkSession

def flatten_json(json_obj, prefix=''):
    flattened = {}
    for key, value in json_obj.items():
        new_key = prefix + key
        if isinstance(value, dict):
            flattened.update(flatten_json(value, new_key + '_'))
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, dict):
                    flattened.update(flatten_json(item, new_key + f'_{i}_'))
                else:
                    flattened[new_key + f'_{i}'] = item
        else:
            flattened[new_key] = value
    return flattened


# Create a Spark session
spark = SparkSession.builder.getOrCreate()

# Load the data as separate JSON objects
json_objects = [json.loads(line) for line in data.strip().split('\n')]

# Flatten the JSON objects
flattened_objects = [flatten_json(obj) for obj in json_objects]

# Convert the flattened objects into a Spark DataFrame
df = spark.createDataFrame(flattened_objects)
df.printSchema()
# Show the flattened DataFrame
df.show()


In [0]:
df_customer =  spark.read.json("/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/customer.json")
df_customer.printSchema()
# df_customer.show()
# Flatten the JSON objects
# flattened_objects = [flatten_json(obj) for obj in df_customer]

# # Convert the flattened objects into a Spark DataFrame
df = flatten_df(df_customer)

df.orderBy(df.customer_id).show()
df.printSchema()




## Flattening deep nested JSON using python and Databricks
[link text](https://sanajitghosh.medium.com/flattening-deep-nested-json-using-python-and-databricks-110d097efa69)

In [0]:
import json
import pandas as pd
from pyspark.sql.functions import explode
from pyspark.sql.functions import col
from pyspark.sql.functions import arrays_zip
from pyspark.sql import functions as F

#load json object
df = spark.read.option("multiline","true").json('/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/data.json')

def child_struct(nested_df):
    # Creating python list to store dataframe metadata
    list_schema = [((), nested_df)]
    # Creating empty python list for final flattern columns
    flat_columns = []

    while len(list_schema) > 0:
      # Removing latest or recently added item (dataframe schema) and returning into df variable
          parents, df = list_schema.pop()
          flat_cols = [  col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],))) for c in df.dtypes if c[1][:6] != "struct"   ]

          struct_cols = [  c[0]   for c in df.dtypes if c[1][:6] == "struct"   ]

          flat_columns.extend(flat_cols)
          #Reading  nested columns and appending into stack list
          for i in struct_cols:
                projected_df = df.select(i + ".*")
                list_schema.append((parents + (i,), projected_df))
    return nested_df.select(flat_columns)


from pyspark.sql.functions import *
def master_array(df):
    array_cols = [c[0] for c in df.dtypes if c[1][:5]=="array"]
    while len(array_cols)>0:
        # array
        for c in array_cols:
           df = df.withColumn(c,explode_outer(c))
        df = child_struct(df)
        array_cols = [c[0] for c in df.dtypes if c[1][:5]=="array"]
    return df

df_output = master_array(df)
display(df_output)
df_output.show(10)



In [0]:
from pyspark.sql.types import *
from pyspark.sql import functions as f

def flatten_structs(nested_df):
    stack = [((), nested_df)]
    columns = []

    while len(stack) > 0:

        parents, df = stack.pop()

        array_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:5] == "array"
        ]

        flat_cols = [
            f.col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],)))
            for c in df.dtypes
            if c[1][:6] != "struct"
        ]

        nested_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:6] == "struct"
        ]

        columns.extend(flat_cols)

        for nested_col in nested_cols:
            projected_df = df.select(nested_col + ".*")
            stack.append((parents + (nested_col,), projected_df))

    return nested_df.select(columns)

def flatten_array_struct_df(df):

    array_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:5] == "array"
        ]


    while len(array_cols) > 0:

        for array_col in array_cols:
            if isinstance(df.schema[array_col].dataType.elementType, StructType) is True:

                cols_to_select = [x for x in df.columns if x != array_col ]

                df = df.withColumn(array_col, f.explode(f.col(array_col)))

        df = flatten_structs(df)

        array_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:5] == "array"
        ]
        return df

flat_df = flatten_array_struct_df(df_product_2)

flat_df.show()

## David et Moi

In [0]:
from urllib.request import urlopen
from pyspark.sql.types import *
from pyspark.sql import functions as f
from urllib.request import urlopen


def flatten_structs(nested_df):
    stack = [((), nested_df)]
    columns = []

    while len(stack) > 0:

        parents, df = stack.pop()

        array_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:5] == "array"
        ]

        flat_cols = [
            f.col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],)))
            for c in df.dtypes
            if c[1][:6] != "struct"
        ]

        nested_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:6] == "struct"
        ]

        columns.extend(flat_cols)

        for nested_col in nested_cols:
            projected_df = df.select(nested_col + ".*")
            stack.append((parents + (nested_col,), projected_df))

    return nested_df.select(columns)

def flatten_array_struct_df(df):

    array_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:5] == "array"
        ]


    while len(array_cols) > 0:

        for array_col in array_cols:
            # check if array contains structure then flatten
            if isinstance(df.schema[array_col].dataType.elementType, StructType) is True:

                cols_to_select = [x for x in df.columns if x != array_col ]

                df = df.withColumn(array_col, f.explode(f.col(array_col)))

        df = flatten_structs(df)

        array_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:5] == "array"
        ]
        return df


url = 'https://world.openfoodfacts.org/api/v2/product/737628064502.json'
jsonData = urlopen(url).read().decode('utf-8')
rdd = spark.sparkContext.parallelize([jsonData])
df_link = spark.read.json(rdd, multiLine=True)

#load json object
df = spark.read.option("multiline","true").json('/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/data.json')

df_link_fl = flatten_structs(df)
df_link_fl.show()
df_link_fl = flatten_array_struct_df(df_link_fl)
df_link_fl.show()

In [0]:
from urllib.request import urlopen
from pyspark.sql.types import *
from pyspark.sql import functions as f
from urllib.request import urlopen


def flatten_structs(nested_df):
    stack = [((), nested_df)]
    columns = []

    while len(stack) > 0:

        parents, df = stack.pop()

        array_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:5] == "array"
        ]

        flat_cols = [
            f.col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],)))
            for c in df.dtypes
            if c[1][:6] != "struct"
        ]

        nested_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:6] == "struct"
        ]

        columns.extend(flat_cols)

        for nested_col in nested_cols:
            projected_df = df.select(nested_col + ".*")
            stack.append((parents + (nested_col,), projected_df))

    return nested_df.select(columns)

def flatten_array_struct_df(df):

    array_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:5] == "array"
        ]


    while len(array_cols) > 0:

        for array_col in array_cols:
            # check if array contains structure then flatten
            if isinstance(df.schema[array_col].dataType.elementType, StructType) is True:

                cols_to_select = [x for x in df.columns if x != array_col ]

                df = df.withColumn(array_col, f.explode(f.col(array_col)))

        df = flatten_structs(df)

        array_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:5] == "array"
        ]
        return df



df = spark.read.json("/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/R_0Bd3pzvYZ7Jy0KZ.json", multiLine=True)
# df = spark.read.json("/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/qualtrics.json", multiLine=True)
# df.printSchema()
# df.show(10)

df_link_fl = flatten_structs(df)
df_link_fl.show()
df_link_fl = flatten_array_struct_df(df_link_fl)
df_link_fl.show()